In [1]:
import os, json, re
import random, time
import pandas as pd, numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta
from dotenv import load_dotenv

from itertools import islice

# Util built for sharktank project
import sharktank_utils_mod as su
from sharktank_utils_mod import load_facts, PitchOrchestrator, metrics_calculation, PitchEditor, OrchestratorResponse, PitchEquipped

# Agno for llm agents
from agno.agent import Agent
from agno.models.groq import Groq
from agno.storage.agent.sqlite import SqliteAgentStorage
from agno.tools.duckduckgo import DuckDuckGoTools
# from agno.tools.wikipedia import WikipediaTools

# Other LLM utils
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Dict, Optional

In [2]:
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
reference_models = su.REFERENCE_MODELS
agent_storage = su.agent_storage
edit_reference_models = su.EDIT_REFERENCE_MODELS

In [4]:
facts_store = load_facts()
pitches = {k:"" for k,_ in facts_store.items()}
metrics = {k:"" for k,_ in facts_store.items()}
time_stamps = {k:"" for k,_ in facts_store.items()}

In [5]:
# Main loop
orcheditor = PitchEquipped(reference=edit_reference_models, iterations=2)
goal = 'create a pitch with the given facts'

RUN_FIRST_N = True      # Change to False to run all
N = 1

cases_to_run = islice(facts_store.items(), N) if RUN_FIRST_N else facts_store.items()
for case, facts in tqdm(cases_to_run, desc="Generating pitches"):
    # case = 'facts_shark_tank_transcript_37_TIK PIK.txt'
    lean_facts = {
        'facts': facts['facts'],
        'product_description': facts['product_description']
    }
    try:
        pitch = orcheditor.orchestrate_with_edit(goal, lean_facts, verbose=False)
        pitches[case] = pitch
        metrics[case] = orcheditor.logs
        time_stamps[case] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    except Exception as e:
        print(f"{case} error: {e}")
        pitches[case] = ""
        metrics[case] = []
        time_stamps[case] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Reset for next run
    orcheditor.logs = []
    orcheditor.agents = {}

[00:48<00:00] 2/2 | Running Pitch Editing Iterations: 100%|██████████ 24.30s/it
Generating pitches: 1it [00:48, 48.60s/it]


In [6]:
pitches.keys()

dict_keys(['facts_shark_tank_transcript_0_GarmaGuard.txt', 'facts_shark_tank_transcript_0_Roadie.txt', 'facts_shark_tank_transcript_0_TouchUp Cup.txt', 'facts_shark_tank_transcript_10_Deux.txt', 'facts_shark_tank_transcript_10_Fish Fixe.txt', 'facts_shark_tank_transcript_10_Hidrent.txt', 'facts_shark_tank_transcript_11_Dude Wipes.txt', 'facts_shark_tank_transcript_11_Love & Pebble.txt', 'facts_shark_tank_transcript_11_Pink Picasso.txt', 'facts_shark_tank_transcript_11_Sheets Laundry Club.txt', 'facts_shark_tank_transcript_11_Zach & Zoe Sweet Bee Farm.txt', 'facts_shark_tank_transcript_12_Elf Grams.txt', 'facts_shark_tank_transcript_12_Ornament Anchor.txt', "facts_shark_tank_transcript_12_Santa's Enchanted Mailbox.txt", "facts_shark_tank_transcript_12_Wendy's Gnome Shop.txt", 'facts_shark_tank_transcript_13_Banana Loca.txt', 'facts_shark_tank_transcript_13_Liberate.txt', 'facts_shark_tank_transcript_13_MAXPRO SmartConnect.txt', 'facts_shark_tank_transcript_13_Tenikle.txt', 'facts_shark_

In [7]:
print(pitches["facts_shark_tank_transcript_0_GarmaGuard.txt"])

{
    "Pitch": "We are introducing GarmaGuard, a revolutionary, eco-friendly product that offers a unique solution for outdoor enthusiasts and environmentally conscious consumers. With a strong marketing and product development plan in place, we are poised to disrupt the market and capture a significant share. Our competitive research has identified a gap in the market for a product like GarmaGuard, and we are confident that our digital marketing campaigns, influencer partnerships, and content marketing strategies will drive growth and adoption. We believe that GarmaGuard has the potential to become a household name and are seeking funding to support our launch and expansion plans.",
    "Initial_Offer": {
        "Valuation": "$5 million",
        "Equity_Offered": "10%",
        "Funding_Amount": "$500,000",
        "Key_Terms": "The funding will be used to support product development, marketing, and operational costs. Investors will have a seat on the board and regular updates on th

In [8]:
print(pitches["facts_shark_tank_transcript_0_Roadie.txt"])